<a href="https://colab.research.google.com/github/bhadreshpsavani/UnderstandingNLP/blob/master/CheckingDeepspeed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We want to check slowdown in while using fp16 instead fp32 while evaluation step.
https://github.com/huggingface/transformers/issues/10816

In [1]:
!git clone https://github.com/huggingface/transformers
%cd transformers
!pip install .
!pip install sacrebleu
!pip install -q sentencepiece
!pip install -r ./examples/question-answering/requirements.txt
%cd ..

Cloning into 'transformers'...
remote: Enumerating objects: 79, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 68149 (delta 26), reused 44 (delta 10), pack-reused 68070
Receiving objects: 100% (68149/68149), 51.86 MiB | 29.67 MiB/s, done.
Resolving deltas: 100% (48151/48151), done.
/content/transformers
Processing /content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.2MB 17.6MB/s 
     |████████████████████████████████| 890kB 54.0MB/s 
  Created wheel for transformers: filename=transformers-4.5.0.dev0-cp37-none-any.whl size=2037253 sha256=8f7f03b11738195b78135cf2022503313a2cb9a9fc9502c61447cfe8466be837
  Stored in directory: /tmp/pip-ephem-wheel-cache-ut9ewt4a/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
Successfully built transformers
  Created wheel for sacremos

In [2]:
!python ./transformers/examples/seq2seq/run_translation.py --model_name_or_path t5-small --output_dir content/tmp/zero3 \
--overwrite_output_dir --max_train_samples 10 --max_val_samples 100 --max_source_length 12 \
--max_target_length 128 --val_max_target_length 128 --do_train --num_train_epochs 1 \
--per_device_train_batch_size 2 --learning_rate 3e-3 --warmup_steps 8 --predict_with_generate \
--logging_steps 0 --save_steps 2 --eval_steps 1 --group_by_length --adafactor --dataset_name wmt16 \
--dataset_config ro-en --source_lang en --target_lang ro \
--source_prefix "translate English to Romanian: " --do_eval 

2021-03-30 17:36:50.764185: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
03/30/2021 17:36:51 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
03/30/2021 17:36:51 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(output_dir='content/tmp/zero3', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=<IntervalStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=2, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=0.003, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1.0, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, warmup_ratio=0.0, warmup_steps=8, logging_dir='run

In [3]:
!python ./transformers/examples/seq2seq/run_translation.py --model_name_or_path t5-small --output_dir /tmp/zero3 \
--overwrite_output_dir --max_train_samples 10 --max_val_samples 100 --max_source_length 12 \
--max_target_length 128 --val_max_target_length 128 --do_train --num_train_epochs 1 \
--per_device_train_batch_size 2 --learning_rate 3e-3 --warmup_steps 8 --predict_with_generate \
--logging_steps 0 --save_steps 2 --eval_steps 1 --group_by_length --adafactor --dataset_name wmt16 \
--dataset_config ro-en --source_lang en --target_lang ro \
--source_prefix "translate English to Romanian: " --do_eval  \
--fp16_full_eval

2021-03-30 17:38:45.642559: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
03/30/2021 17:38:46 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
03/30/2021 17:38:46 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(output_dir='/tmp/zero3', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=<IntervalStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=2, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=0.003, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1.0, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, warmup_ratio=0.0, warmup_steps=8, logging_dir='runs/Mar30

In [6]:
# how much more time it takes during evulation while using fp16?
(14.87/11.58 - 1)*100

28.41105354058722

## Conclusion:
It takes around 28% more time when we use FP16 flag